In [3]:
print("ASSIGNMENT - 3 (QUESTION : 6(D))\n")
print("Mitta Lekhana Reddy\n")
print("IIT2019204")

ASSIGNMENT - 3 (QUESTION : 6(D))

Mitta Lekhana Reddy

IIT2019204


In [40]:
import numpy as np
import pandas as pd
import random
import math

In [42]:
url = 'https://raw.githubusercontent.com/LekhanaMitta/IML-Assign2-Q6/main/Housing_Price_data_set%5B1%5D.csv'
plan = pd.read_csv(url)

Y = plan['price']
plotarea = plan['plotsize']
bedroom_no = plan['bedrooms']
bathroom_no = plan['bathrooms']
Y = np.array(Y)
Y = Y.reshape(Y.shape[0], 1)

In [36]:
def calculate_error(Y, Y_pred):
    error = 0
    for i in range(len(Y)):
        error += abs(Y[i] - Y_pred[i]) / Y[i]
    error = error / len(Y)
    return error * 100

In [37]:
def kernel(X, xi, Tau):
    return np.exp(-np.sum((xi - X) ** 2, axis = 1) / (2 * Tau * Tau))

In [22]:
def LocallyWeightedLR(X, xi, Y, Tau):
	X_ = np.transpose(X)
	W = kernel(X, xi, Tau)
	XW_ = X_ * W
	XW_X = np.matmul(XW_, X)
	_XW_X = np.linalg.pinv(XW_X)
	_XW_XXW_ = np.matmul(_XW_X, XW_)
	_XW_XXW_Y = np.matmul(_XW_XXW_, Y)
	_XW_XXW_Y_ = np.transpose(_XW_XXW_Y)
	return _XW_XXW_Y_.dot(xi)

**Perform Features scanning on PlotArea :**

In [38]:
mean_plotarea = np.mean(plotarea)
max_plotarea = max(plotarea)
min_plotarea = min(plotarea)
plotarea_Scaled = []
for i in plotarea:
	plotarea_Scaled.append((i - mean_plotarea) / (max_plotarea - min_plotarea))

**Using Locally Weighted Linear Regression for Tau :**

In [39]:
X = []
for i in range(len(plotarea)):
	X.append([1, plotarea_Scaled[i], bedroom_no[i], bathroom_no[i]])
X = np.array(X)

Tau = 0.00005
print("Using Locally Weighted Linear Regression for Tau = " + str(Tau))
pred = []
for i in range(X.shape[0]):
	y_pred = LocallyWeightedLR(X, X[i], Y, Tau)
	pred.append(y_pred)
print("Mean absolute percentage error is : " + str(calculate_error(Y,pred)))
print()

cost = plan['price']

Using Locally Weighted Linear Regression for Tau = 5e-05
Mean absolute percentage error is : [5.40732082]



**Segmenting the Features :**

In [26]:
t_features = []
for i in range(383):
	t_features.append([1, plotarea_Scaled[i], bedroom_no[i], bathroom_no[i]])
t_cost = cost[:383]
cost_test = []
test_features = []
for i in range(383, len(cost)):
	test_features.append([1, plotarea_Scaled[i], bedroom_no[i], bathroom_no[i]])
	cost_test.append(cost[i])
m = len(t_features)
print("No. of features is : ")
print(m)

No. of features is : 
383


In [27]:
def Slope(Coeff, t_features, t_cost, ind):
	Error = 0
	for i in range(len(t_features)):
		itr = 0
		for j in range(len(Coeff)):
			itr = itr + Coeff[j] * t_features[i][j]
		Error += (itr - t_cost[i]) * t_features[i][ind]
	return Error

**Using Scaled Batch Gradient with Regularisation :**

In [28]:
print("Using scaled batch gradient with regularisation")
L_Rate = 0.001
Lambda_Param = -49
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)
for epochs in range(5000):
	Temp_Coeff = Coeff.copy()
	for j in range(len(Coeff)):
		if (j == 0):
			Temp_Coeff[j] = Temp_Coeff[j] - ((L_Rate / m) * (Slope(Coeff, t_features, t_cost, j)))	
		else:
			Temp_Coeff[j] = (1 - L_Rate * Lambda_Param / m) * Temp_Coeff[j] - ((L_Rate / m) * (Slope(Coeff, t_features, t_cost, j)))
	Coeff = Temp_Coeff.copy()
print("Final coefficients are:")
print(Coeff)

Using scaled batch gradient with regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[5037.585668619078, 11147.667574879839, 10378.580439168689, 22647.298983883848]


**Finding Mean Absolute percentage error for Batch Gradient Regularisation :**

In [29]:
Error = 0
for i in range(len(test_features)):
	predict = 0
	for j in range(len(Coeff)):
	  	predict = predict + Coeff[j] * test_features[i][j]
	Error += abs(predict - cost_test[i]) / cost_test[i]
Error = (Error / len(test_features)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

Mean absolute percentage error is : 19.92701396456417



In [30]:
def SlopeStoch(Coeff,t_features,Act_Val,ind):
	itr = 0
	for j in range(len(Coeff)):
		itr = itr + Coeff[j]*t_features[j]
	return (itr - Act_Val) * t_features[ind]

**Using Scaled Stochastic Gradient with regularisation :**

In [31]:
print("Using Stochastic gradient with regularisation")

L_Rate = 0.005
Lambda_Param = 142
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)

for iter in range(10):
	for i in range(len(t_cost)):
		Temp_Coeff = Coeff.copy()
		for j in range(4):
			if j == 0:
				Temp_Coeff[j] = Temp_Coeff[j] - (L_Rate * (SlopeStoch(Coeff, t_features[i], t_cost[i], j)))
			else:
				Temp_Coeff[j] = (1 - L_Rate * Lambda_Param) * Temp_Coeff[j] - (L_Rate * (SlopeStoch(Coeff, t_features[i], t_cost[i], j)))
		Coeff = Temp_Coeff.copy()

print("Final coefficients are:")
print(Coeff)

Using Stochastic gradient with regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[69852.81456771938, 196.16376268823387, 1257.4622739827178, 431.8951419159166]


**Finding Mean Absolute percentage error for Stochastic Gradient Regularisation :**

In [32]:
# Finding Mean absolute percentage error.
Error = 0
for i in range(len(test_features)):
	predict = 0
	for j in range(len(Coeff)):
	  	predict = predict + Coeff[j] * test_features[i][j]
	Error += abs(predict - cost_test[i]) / cost_test[i]
Error = (Error / len(test_features)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

Mean absolute percentage error is : 22.675518794902167



**Using Scaled Minibatch Gradient with Regularisation for batch size = 20 :**

In [33]:
print("Using Scaled Minibatch gradient with regularisation for batch size = 20")

Batch = 20;
L_Rate = 0.002
Lambda = -372
Coeff = [0, 0, 0, 0]
batch_no = math.ceil(len(t_cost) / Batch)
equal_div = False
if (len(t_cost) % Batch == 0):
	equal_div = True;

for epoch in range(30):
	for batch in range(batch_no):
		Sum = [0, 0, 0, 0]
		for j in range(len(Coeff)):
			for i in range(Batch):
				if (batch * Batch + i == len(t_features)):
					break
				Predict = 0.0
				for wj in range(len(Coeff)):
					Predict += Coeff[wj] *t_features[batch * Batch + i][wj]
				Predict -= t_cost[batch * Batch + i]
				Predict *= t_features[batch * Batch + i][j]
				Sum[j] += Predict;

		if (not equal_div and batch == batch_no - 1):
			for j in range(len(Sum)):
				if j == 0:
					Coeff[j] -= (Sum[j] / (len(t_cost) % Batch)) * L_Rate
				else:
					Coeff[j] = (1 - L_Rate * Lambda / m) * Coeff[j] - (Sum[j] / (len(t_cost) % Batch)) * L_Rate
		else:
			for j in range(len(Sum)):
				if j == 0:
					Coeff[j] -= (Sum[j] / Batch) * L_Rate
				else:
					Coeff[j] = (1 - L_Rate * Lambda / m) * Coeff[j] - (Sum[j] / Batch) * L_Rate
print("Final coefficients are:")
print(Coeff)

Using Scaled Minibatch gradient with regularisation for batch size = 20
Final coefficients are:
[888.9201243627547, 5168.211726125255, 17701.360814619125, 15202.387873756412]


**Finding Mean Absolute percentage Error for Scaled Minibatch Gradient Regularisation :**

In [34]:
Error = 0
for i in range(len(test_features)):
	predict = 0
	for j in range(len(Coeff)):
	  	predict = predict + Coeff[j] * test_features[i][j]
	Error += abs(predict - cost_test[i]) / cost_test[i]
Error = (Error / len(test_features)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

Mean absolute percentage error is : 19.550681895981263

